## 로보 플로우 데이터셋

In [3]:
from roboflow import Roboflow

rf = Roboflow(api_key="TBDNbDNNjBRxOoBGiu48")
project = rf.workspace("mokhamed-nagy-u69zl").project("furniture-detection-qiufc")
dataset = project.version(20).download("yolov8")


ModuleNotFoundError: No module named 'roboflow'

## 파일 관련 (이동, 갯수 계산, 사이즈 변환)

In [ ]:
#전체 폴더 이동
import os
import shutil

source_folder = '/content/furniture-detection-20'  # 이동시킬 원본 폴더
destination_folder = '/content/drive/MyDrive/Colab Notebooks/BigP/dataset'  # 파일을 이동할 대상 폴더

# 대상 폴더가 없으면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 원본 폴더의 모든 내용(파일 및 하위 폴더)을 대상 폴더로 이동
for item in os.listdir(source_folder):
    source_path = os.path.join(source_folder, item)
    destination_path = os.path.join(destination_folder, item)

    # 이동
    shutil.move(source_path, destination_path)

In [ ]:
#파일 이동
import os
import shutil

source_folder = '/content/runs/detect/train/weights'
destination_folder = '/content/drive/MyDrive/Colab Notebooks/BigP/weights/x_e100'

# 대상 폴더가 없으면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for filename in os.listdir(source_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    # 파일 이동
    shutil.move(source_path, destination_path)

In [ ]:
#파일 수 계산
import os
def count_files(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

aa = '/content/drive/MyDrive/Colab Notebooks/BigP/dataset/valid'  # 폴더 경로 지정
folder_path = f'{aa}/images'  # 폴더 경로 지정
folder_path2 = f'{aa}/labels'  # 폴더 경로 지정
print(f"images 파일 수: {count_files(folder_path)}")
print(f"labels 파일 수: {count_files(folder_path2)}")

images 파일 수: 891
labels 파일 수: 891


In [ ]:
# 이미지 크기 조절
from ultralytics import YOLO
from PIL import Image
import os, torch

folder_path = '/content/'
for filename in os.listdir(folder_path):
    if filename.endswith('.png'):
        img_path = os.path.join(folder_path, filename)
        with Image.open(img_path) as img:
            # 이미지 크기 조절
            img_resized = img.resize((640, 640))

            # 변경된 이미지 저장
            img_resized.save(img_path)

## 로보 플로우 모델

In [1]:
#로보 플로우 모델
from roboflow import Roboflow
rf = Roboflow(api_key="TBDNbDNNjBRxOoBGiu48")
project = rf.workspace().project("furniture-detection-qiufc")
model = project.version(20).model

loading Roboflow workspace...
loading Roboflow project...


In [2]:
#로보 플로우 예측
import os

imgJson = []
folder_path = 'C:\\Users\\user\\Desktop\\BigF\\team5\\Detection\\test_crawling_data'
save_predict = 'C:\\Users\\user\\Desktop\\BigF\\team5\\Detection\\yolo\\predict_img'
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for fn in file_names:
    model.predict(f"{folder_path}\\{fn}", confidence=50, overlap=20).save(f'{save_predict}\\pred_{fn}')
    imgJson.append(model.predict(f"{folder_path}\\{fn}", confidence=50, overlap=20).json())

HTTPError: 403 Client Error: Forbidden for url: https://detect.roboflow.com/furniture-detection-qiufc/20?api_key=TBDNbDNNjBRxOoBGiu48&name=YOUR_IMAGE.jpg&overlap=20&confidence=50&stroke=1&labels=false&format=json

## 이미지 크롭

#### 박스만 남기기

In [10]:
import cv2

save_crop = f'C:\\Users\\user\\Desktop\\BigF\\team5\\Detection\\yolo\\crop_img'

for n in range(len(file_names)):
    full_path = f"{folder_path}\\{file_names[n]}"    
    img = cv2.imread(full_path)
        
    boxes = []
    NN = len(imgJson[n]['predictions'])
    for N in range(NN): # N번째 박스
        print(imgJson[n]['predictions'][N])
        xh = int(imgJson[n]['predictions'][N]['width']/2)
        yh = int(imgJson[n]['predictions'][N]['height']/2)
        x1 = imgJson[n]['predictions'][N]['x'] - xh
        y1 = imgJson[n]['predictions'][N]['y'] - yh
        x2 = imgJson[n]['predictions'][N]['x'] + xh
        y2 = imgJson[n]['predictions'][N]['y'] + yh
        boxes.append([x1, y1, x2, y2])
    
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        print(box, x1, y1, x2, y2)
        # 바운딩 박스 영역 잘라내기
        cropped_img = img[y1:y2, x1:x2]
        cv2.imwrite(f'{save_crop}\\crop_{file_names[n]}_{i}.jpg', cropped_img)
        print(f"Saved cropped image")

{'x': 789, 'y': 950, 'width': 1420, 'height': 955, 'confidence': 0.6043277382850647, 'class': 'Table', 'class_id': 20, 'image_path': 'C:\\Users\\user\\Desktop\\BigF\\team5\\Detection\\test_crawling_data\\AAIRLLENComputerDeskInchModernSimpleSturdyNotebookWritingDeskStudyOfficeTableforHomeOfficeDinningTableWorkstationDeskEscritoriodeOrdenador.jpg', 'prediction_type': 'ObjectDetectionModel'}
[79, 473, 1499, 1427] 79 473 1499 1427
Saved cropped image
{'x': 919, 'y': 1191, 'width': 1153, 'height': 616, 'confidence': 0.6365021467208862, 'class': 'Wooden floor', 'class_id': 24, 'image_path': 'C:\\Users\\user\\Desktop\\BigF\\team5\\Detection\\test_crawling_data\\AAIRLLENSleekandSturdyInchComputerDeskModernDesignforHomeOfficePerfectforWorkandStudyMultiPurposeTableforWritingDiningandWorkstation.jpg', 'prediction_type': 'ObjectDetectionModel'}
[343, 883, 1495, 1499] 343 883 1495 1499
Saved cropped image
{'x': 953, 'y': 1146, 'width': 1063, 'height': 692, 'confidence': 0.6950677633285522, 'class':

#### 박스 제외 지우기

In [4]:
# import cv2

# # 이미지 로드
# img = cv2.imread(full_path)

# # 객체 탐지 후, 바운딩 박스 좌표 예시
# boxes = [[x1, y1, x2, y2], ...]

# # 이미지의 복사본 생성
# mask = np.zeros_like(img)

# for box in boxes:
#     x1, y1, x2, y2 = box
#     # 바운딩 박스 영역을 복사본에 복사
#     mask[y1:y2, x1:x2] = img[y1:y2, x1:x2]

# # 결과 이미지 보기
# cv2.imshow("Masked Image", mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

{'x': 719, 'y': 289, 'width': 1438, 'height': 578, 'confidence': 0.8438215255737305, 'class': 'Sofa', 'class_id': 18, 'image_path': 'C:\\Users\\user\\Desktop\\Detector\\yolo\\test_crawlingdata\\CasaAndreaMilanoModernVelvetFabricSectionalSofaLShapeCouchwithExtraWideChaiseLounge.jpg', 'prediction_type': 'ObjectDetectionModel'}
719 289 2157 867
